In [1]:
import re
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_similarity
from itertools import combinations

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/saifhaddad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saifhaddad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df=pd.read_csv('preprocessed_documents.csv')

df.head(10)

CASE-DESC
0  عدم التصفح وسوء الخدمه والرجاء اتخاذ قرار سكان...
1  activ date sep 22 2022 يتم اضافه الحزم رقم الت...
2  never connect ont unknowncontact number 077715...
3       never connect ontunknown07994746460790205560
4         never connect ontاعاده الاتصال جانب المودم
5  never connect ontالمشترك رافع السرعه وتم الدخو...
6  never connect ontالمشترك يستطيع الدخول للموقع ...
7  never connect ontمشكله برمجيه علي الموديممسجله...
8                                       reduc robust
9  reduc robust minimum 2921 dbmmaximum 2824 dbma...

In [3]:
print(df.columns)


Index(['CASE-DESC'], dtype='object')


In [4]:
# Define Arabic and English punctuations
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

# Define Arabic stopwords
stop_words = set(stopwords.words('arabic'))

# Regular expression for Arabic
arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

In [5]:
preprocessed_documents = df['CASE-DESC'].tolist()


In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_similarity

global global_results
global_results = None

Query = None

def generater():
    global Query
    global global_results  # Declare the global variable

    Query = entry_query.get()
    #num_results = entry_num_results.get()

        # Preprocessing function
    def preprocess_text(text):
    
        # Remove punctuations
        translator = str.maketrans('', '', punctuations)
        text = text.translate(translator)

        # Remove Arabic 7arakat
        text = re.sub(arabic_diacritics, '', text)

        # Remove punctuations
        text = ''.join([char for char in text if char not in punctuations])

        # Normalize some Arabic characters
        text = re.sub("[إأآا]", "ا", text)
        text = re.sub("ى", "ي", text)
        text = re.sub("ؤ", "ء", text)
        text = re.sub("ئ", "ء", text)
        text = re.sub("ة", "ه", text)
        text = re.sub("گ", "ك", text)

        # Tokenization
        tokens = nltk.word_tokenize(text)

        # Lowercase conversion for English tokens
        tokens = [token.lower() for token in tokens]

        # Stopword removal for both English and Arabic
        stop_words = set(stopwords.words('english')) | set(stopwords.words('arabic'))
        tokens = [token for token in tokens if token not in stop_words]

        # Punctuation removal
        tokens = [token for token in tokens if re.match(r'\w', token)]

        # Stemming (Note: This is only for English; Arabic would need a different stemmer)
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]

        # Joining tokens back into text
        processed_text = ' '.join(tokens)
        return processed_text
    
    preprocessed_query = preprocess_text(Query)

    print(preprocessed_query)
    
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity   
    
    
    vectorizer = TfidfVectorizer()
    tfidf_documents = vectorizer.fit_transform(preprocessed_documents)
    tfidf_query = vectorizer.transform([preprocessed_query])

    preprocessed_df = pd.DataFrame(preprocessed_documents, columns=['Preprocessed CASE_DESC'])
    
    
    # Calculate similarity metrics
    euclidean_dist = euclidean_distances(tfidf_query, tfidf_documents).flatten()
    manhattan_dist = manhattan_distances(tfidf_query, tfidf_documents).flatten()
    cosine_dist = cosine_similarity(tfidf_query, tfidf_documents).flatten()
    
    # Create results DataFrame
    results_df = pd.DataFrame({
        "Preprocessed CASE_DESC": preprocessed_documents,
        "Euclidean Distances": euclidean_dist,
        "Manhattan Distances": manhattan_dist,
        "Cosine Similarity": cosine_dist
    })
    sorted_cosine = results_df.sort_values(by='Cosine Similarity', ascending=False)
    

    # Get the number of results to display
    try:
        num_results = int(entry_num_results.get())
    except ValueError:
        num_results = 5  # Default number of results if input is invalid


    # Create a new top-level window
    result_window = tk.Toplevel(root)
    result_window.title("Results")
    
    # Display the results in the new window
    results_text = "\n".join(sorted_cosine['Preprocessed CASE_DESC'].head(num_results).tolist())
    label_results = tk.Label(result_window, text=results_text, justify=tk.LEFT)
    label_results.pack(pady=10)

    # Add an exit button to the result window
    exit_button_result_window = tk.Button(result_window, text="Exit", command=result_window.destroy)
    exit_button_result_window.pack(pady=10)
    sorted_descriptions = sorted_cosine['Preprocessed CASE_DESC'].head(num_results)

    global_results = sorted_descriptions


    


In [7]:
import tkinter as tk

# Create the main window
root = tk.Tk()

# Set the title of the window
root.title("Case Descriptions")

# set the window size
root.geometry("600x500")

# Add a Label widget
label_query = tk.Label(root, text="Insert query:")
label_query.pack(pady=10)

# Add a Text widget 
entry_query = tk.Entry(root)
entry_query.pack(pady=10)

# Number of results input
label_num_results = tk.Label(root, text="Number of results to display:")
label_num_results.pack(pady=10)

entry_num_results = tk.Entry(root)
entry_num_results.pack(pady=10)


# Add a Button widget
submit_button = tk.Button(root, text="Generate", command=generater)
submit_button.pack()

exit_button_main_window = tk.Button(root, text="Exit", command=root.destroy)
exit_button_main_window.pack(pady=10)


# Run the application
root.mainloop()


مشكله الانترنت


In [8]:
print(global_results)

118605             مشكله الانترنت
114185        يوجد مشكله الانترنت
118643        مشكله شبكه الانترنت
114176    يوجد مشكله علي الانترنت
125575       يعاني مشكله الانترنت
Name: Preprocessed CASE_DESC, dtype: object


In [9]:
import tkinter as tk
from tkinter import messagebox, Toplevel
from PIL import Image, ImageTk
import networkx as nx
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import arabic_reshaper
from bidi.algorithm import get_display


In [10]:
def generate_directed_graph():
    global global_results
    
    # Create a directed graph
    G = nx.DiGraph()

    # Tokenize each document and add words as nodes
    for document in global_results:
        words = word_tokenize(document)
        reshaped_words = [get_display(arabic_reshaper.reshape(word)) for word in words]  # Reshape Arabic words
        G.add_nodes_from(reshaped_words)

        # Add directed edges between adjacent words
        for i in range(len(reshaped_words) - 1):
            G.add_edge(reshaped_words[i], reshaped_words[i+1])

    pagerank = nx.pagerank(G)
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)

    # Visualization
    plt.figure(figsize=(10, 8))
    pos = nx.circular_layout(G)
    nx.draw(G, pos, with_labels=True, arrows=True, arrowsize=10, node_size=800, node_color="skyblue", font_size=10)
    plt.savefig("graph.png")
    plt.close()

    # Create a new top-level window for displaying the graph
    graph_window = Toplevel(root)
    graph_window.title("Directed Graph Results")
    graph_window.geometry("1000x800") 

    
    # Load and display the image in the new window
    try:
        img = Image.open("graph.png")
        img = img.resize((1000, 800), Image.ANTIALIAS)  # image size
        img = ImageTk.PhotoImage(img)
        graph_panel = tk.Label(graph_window, image=img)  # Create a label in the new window
        graph_panel.image = img
        graph_panel.pack(pady=10)
    except IOError:
        messagebox.showinfo("Error", "Graph image could not be loaded.", parent=graph_window)


    print("PageRank:", pagerank)
    print("Degree Centrality:", degree_centrality)
    print("Betweenness Centrality:", betweenness_centrality)
    print("Closeness Centrality:", closeness_centrality)


In [11]:
def reshape_arabic(text):
    return get_display(arabic_reshaper.reshape(text))


def generate_weighted_graph():
    global global_results
    

    # Use global_results for generating the graph
    complaints = global_results

    G = nx.Graph()
    co_occurrences = {}

    for complaint in complaints:
        words = word_tokenize(complaint)
        reshaped_words = [reshape_arabic(word) for word in words]
        unique_words = set(reshaped_words)

        for word1, word2 in combinations(unique_words, 2):
            pair = tuple(sorted([word1, word2]))
            if pair in co_occurrences:
                co_occurrences[pair] += 1
            else:
                co_occurrences[pair] = 1

    for pair, weight in co_occurrences.items():
        G.add_edge(pair[0], pair[1], weight=weight)

    plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=2500, font_size=10)
    nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): d['weight'] for u, v, d in G.edges(data=True)}, font_color='red')
    plt.title("Word Co-occurrence Graph")
    plt.savefig("weighted_graph.png")
    plt.close()

    graph_window = Toplevel(root)
    graph_window.title("Weighted Graph Results")
    graph_window.geometry("1110x800")

    try:
        img = Image.open("weighted_graph.png")
        img = img.resize((1000, 800), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        graph_panel = tk.Label(graph_window, image=img)
        graph_panel.image = img
        graph_panel.pack(pady=10)
    except IOError:
        messagebox.showinfo("Error", "Graph image could not be loaded.", parent=graph_window)


In [12]:
# Create the main window
root = tk.Tk()
root.title("Graph Mining")
root.geometry("600x500")  # Main window size

# Add a Label widget
label_graph_mining = tk.Label(root, text="Directed Graph Mining")
label_graph_mining.pack(pady=10)

# Add a Button widget to generate the directed graph
generate_button = tk.Button(root, text="Generate Directed Graph", command=generate_directed_graph)
generate_button.pack(pady=10)


# Add a Label widget for the weighted graph
label_weighted_graph = tk.Label(root, text="Weighted Graph Mining")
label_weighted_graph.pack(pady=10)

# Add a Button widget to generate the weighted graph
generate_weighted_button = tk.Button(root, text="Generate Weighted Graph", command=generate_weighted_graph)
generate_weighted_button.pack(pady=10)


# Add an exit button to the main window
exit_button_main_window = tk.Button(root, text="Exit", command=root.destroy)
exit_button_main_window.pack(pady=10)

# Run the application
root.mainloop()

/var/folders/sb/6phfnpsd4vbd52mr5kht86kw0000gn/T/ipykernel_97100/2002293712.py:38: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((1000, 800), Image.ANTIALIAS)  # image size


PageRank: {'ﻪﻠﻜﺸﻣ': 0.20776393262479373, 'ﺖﻧﺮﺘﻧﻻﺍ': 0.36670324993740744, 'ﺪﺟﻮﻳ': 0.07694987157780299, 'ﻪﻜﺒﺷ': 0.1358165371410964, 'ﻲﻠﻋ': 0.1358165371410964, 'ﻲﻧﺎﻌﻳ': 0.07694987157780299}
Degree Centrality: {'ﻪﻠﻜﺸﻣ': 1.0, 'ﺖﻧﺮﺘﻧﻻﺍ': 0.6000000000000001, 'ﺪﺟﻮﻳ': 0.2, 'ﻪﻜﺒﺷ': 0.4, 'ﻲﻠﻋ': 0.4, 'ﻲﻧﺎﻌﻳ': 0.2}
Betweenness Centrality: {'ﻪﻠﻜﺸﻣ': 0.30000000000000004, 'ﺖﻧﺮﺘﻧﻻﺍ': 0.0, 'ﺪﺟﻮﻳ': 0.0, 'ﻪﻜﺒﺷ': 0.0, 'ﻲﻠﻋ': 0.0, 'ﻲﻧﺎﻌﻳ': 0.0}
Closeness Centrality: {'ﻪﻠﻜﺸﻣ': 0.4, 'ﺖﻧﺮﺘﻧﻻﺍ': 0.7142857142857143, 'ﺪﺟﻮﻳ': 0.0, 'ﻪﻜﺒﺷ': 0.36, 'ﻲﻠﻋ': 0.36, 'ﻲﻧﺎﻌﻳ': 0.0}


/var/folders/sb/6phfnpsd4vbd52mr5kht86kw0000gn/T/ipykernel_97100/4143552559.py:44: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((1000, 800), Image.ANTIALIAS)
